# Overview

In this tutorial we will explore the foundations of box localization networks. First we will examine the most common anchor parameterization of boxes at various scales and ratios across different feature map levels. Second we will explore the a popular backbone common to many modern box localization networks: the feature pyramid network. Finally we will dive into specifies regarding a popular high-performing implementation: RetinaNet.

This tutorial is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found at: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

# Google Colab

The following lines of code will configure your Google Colab environment for this tutorial.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [ ]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries:

In [ ]:
import numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, metrics
from jarvis.train import datasets, custom
from jarvis.train.box import BoundingBox
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of brain tumor MRI exams derived from the MICCAI Brain Tumor Segmentation Challenge (BRaTS). More information about he BRaTS Challenge can be found here: http://braintumorsegmentation.org/. Each single 2D slice will consist of one of four different sequences (T2, FLAIR, T1 pre-contrast and T1 post-contrast). In this exercise, we will use this dataset to derive a model for slice-by-slice tumor segmentation. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/mr_brats_2020`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [ ]:
# --- Download dataset
datasets.download(name='mr/brats-2020-mip')

Once downloaded, the `datasets.prepare(...)` method can be used to generate the required python Generators to iterate through the dataset, as well as a `client` object for any needed advanced functionality.

To specificy the correct Generator template file, pass a designated `keyword` string. In this tutorial, we will be using brain MRI volumes that have been preprocessed using a *mean intensity projection* (MIP) algorithm to subsample the original 155-slice inputs to 40-50 slices, facilitating ease of algorithm training within the Google Colab platform. In addition we will be performing voxel-level tumor prediction (e.g., a prediction for every single voxel in the 3D volume). To select the correct Client template for this task, use the keyword string `mip*vox`. 

Finally, for sake of simplicity, this tutorial will binarize the ground-truth labels (instead of the original four separate tumor classes). To do so, pass the following `configs` dictionary into the `datasets.prepare(...)` method. As needed, modify the custom `configs` dictionary with additional configurations as needed (e.g. batch size, normalization parameters, etc). 

In [ ]:
# --- Prepare generators
configs = {'specs': {'ys': {'tumor': {'norms': {'clip': {'max': 1}}}}}}
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-mip', keyword='mip*vox', configs=configs)

As before, each iteration yields two variables, `xs` and `ys`, each representing a dictionary of model input(s) and output(s). In the current example, there is just a single input and output. Let us examine the generator data:

In [ ]:
# --- Yield one example
xs, ys = next(gen_train)

# --- Print dict keys
print('xs keys: {}'.format(xs.keys()))
print('ys keys: {}'.format(ys.keys()))

In [ ]:
# --- Print data shape
print('xs shape: {}'.format(xs['dat'].shape))
print('ys shape: {}'.format(ys['tumor'].shape))

### 3D operations

Note that the model input shapes for this exercise (and all subsequent exercises) will be provided as 3D tensors. Even if your current model does not require 3D data (as in this current tutorial), all 2D tensors can be represented by a 3D tensor with a z-axis shape of 1. In addition, designing all models with this configuration (e.g. 3D operations) ensures that minimal code changes are needed when testing various 2D and 3D network architectures. 

# Box Parameterization

Recall the common parameterization of boxes across an image using a grid of anchors:

![Box Parameterization](https://raw.githubusercontent.com/peterchang77/dl_tutor/master/cs190/spring_2020/notebooks/box_localization/pngs/box_params.png)

At each anchor location, a total of **A** anchors may be defined spanning a variety of:

* **aspect ratios**: 1:1, 2:1, 1:2, etc ...
* **scales**: 2 ** 0, 2 ** (1/3), 2 ** (2/3), etc ...

For each **A** number of anchors, there are two separate predictions:

* **K**-element logit score representing a binary prediction of whether or not the *k-th* class is present in the box
* **4**-element box fine-tuning representing the shift in the height, width, y- and x-coordinates from base box

### Anchor grid sizes

The H x W size of an anchor grid (e.g. and thus implicitly the correspond box size) is commonly referenced by the **number of subsamples** required relative to the original full image shape. For example, if an original image is (N, N) in shape, then the first subsampled feature map is (N / 2, N / 2), the second subsampled featured map is (N / 4, N / 4), etc...

In this example, the original input images are (240, 240) MR images. Thus the following feature maps (prefixed with `c`) may be defined:

* **c1**: 120 x 120 anchor grid
* **c2**: 60 x 60 anchor grid
* **c3**: 30 x 30 anchor grid
* **c4**: 15 x 15 anchor grid

... and so on. By default, it is most common to start at the `c2` or `c3` level and proceed to include 3 to 5 different resolutions depending on the desired target.

### `BoundingBox`

The `BoundingBox` class as part of the `jarvis-md` library facilitates definition and manipulation of boxes parameterized using the above standard notation. The object initializer has the following arguments:

```
(iter)   image_shape     : original 2D image shape
(int)    classes         : number of non-background classes
(iter)   c               : feature maps to use; c1 = 1st subsample, c2 = 2nd subsample, etc
(iter)   anchor_shapes   : base shape of anchors in each feature map
(iter)   anchor_scales   : scales of each anchor parameterized as 2 ** (i/3)
(iter)   anchor_ratios   : aspect ratios of each anchor
(float)  iou_upper       : upper IoU used for pos boxes
(float)  iou_lower       : lower IoU used for neg boxes
(float)  iou_nms         : IoU used for non-max supression
(int)    box_padding     : padding for ground-truth boxes
(bool)   separate_maps   : if True, create parameters for each feature map separately
```

In [ ]:
# --- Create BoundingBox
bb = BoundingBox(
    image_shape=(240, 240),
    classes=1,
    c=[3, 4],
    anchor_shapes=[32, 64],
    anchor_scales=[0, 1, 2],
    anchor_ratios=[0.5, 1, 2],
    iou_upper=0.5,
    iou_lower=0.2)

This code will initialze all anchors and tempalte boxes based on our specifications:

In [ ]:
# =============================
# PRINT BOUNDING BOX SPECS
# =============================

# --- Print grid sizes
print('---------------------------------------')
print('Anchor Grid Sizes')
print(bb.params['anchor_gsizes'])

# --- Print template anchor box shapes
print('---------------------------------------')
print('Anchor Template Box Shapes')
print(bb.params['anchor_shapes'])

# --- Print anchor details
print('---------------------------------------')
print('Anchor scales: {}'.format(bb.params['anchor_scales']))
print('Anchor ratios: {}'.format(bb.params['anchor_ratios']))

print('---------------------------------------')
print('Total anchors (A) = {}'.format(
    len(bb.params['anchor_scales']) * 
    len(bb.params['anchor_ratios'])))
print('Total classes (k) = {}'.format(bb.params['classes']))

Note that other parameters can be found in `bb.params`.

**Checkpoint**: How many boxes in total are defined by the specifications above?

### Ground-truth

Recall that the ground-truth predictions the box-localization CNN must produce are variable depending on the box parameterization chosen above. Predictions at *multiple feature map resolutions* must be provided for both the classification task (e.g. determine which boxes are positive) and regression task (e.g. determine what modifications are needed to template boxes to create final boxes). 

**Checkpoint**: How many different feature map predictions must the CNN generate in the box parameterization chosen above? What are the shapes for all predicted feature maps? Use `bb.params['inputs_shapes']` to confirm your calculations.

The `BoundingBox` object can create ground-truth box parameterizations using either label masks (e.g. provided in this tutorial) or boxes provided in anchor-style format e.g `[y0, x0, y1, x1]`. To generate ground-truth from a provided mask, use the `bb.convert_msk_to_box(...)` method:

In [ ]:
# --- Create box ground-truths
box = bb.convert_msk_to_box(ys['tumor'][0])

Do the parameterized box ground-truths match your expected tensor shapes?

Raw box parameterizations are difficult to visualize and/or check. Instead to *post-process* box parameterizations, use one of the following methods:

* `bb.convert_box_to_anc(...)`: method to convert box to anchors (e.g. `[y0, x0, y1, x1]`)
* `bb.convert_box_to_msk(...)`: method to convert box to mask label for visualization

For both methods, the `apply_deltas=[True/False]` flag can be used to specify whether or not to apply the box refinements (e.g. regression network predictions).

In [ ]:
# --- Convert box to anchors
anchors, classes = bb.convert_box_to_anc(box, apply_deltas=False)

print('---------------------------------------')
print('\nGround-truth template boxes (before refinement):\n')
print(anchors)

anchors, classes = bb.convert_box_to_anc(box, apply_deltas=True)

print('---------------------------------------')
print('\nGround-truth template boxes (after refinement):\n')
print(anchors)

**Checkpoint**: why are the post-refinement boxes exactly identical?

In [ ]:
# --- Convert box to mask (for visualization)
msk = bb.convert_box_to_msk(box, apply_deltas=False)
imshow(xs['dat'][0, ..., 0], msk, title='Ground-truth template boxes (before refinement)')

In [ ]:
# --- Convert box to mask (for visualization)
msk = bb.convert_box_to_msk(box, apply_deltas=True)
imshow(xs['dat'][0, ..., 0], msk, title='Ground-truth template boxes (after refinement)')

**Checkpoint**: What happens to the appearance of boxes with variations in:

* grid sizes (`c` values)
* anchor shapes
* anchor aspect ratios
* anchor scales

### Generators

To convert all original masks into box parameterizations, pass the existing generators into the `bb.create_generators(...)` method. This will create new generators that utilize the prior generators to load data before applying modifications needed for box parameterization (e.g. nested generators). The `msk=` argument is used to denote the key in the `ys` dictionary containing mask labels to apply box conversion.

In [ ]:
# --- Prepare generators
gen_train, gen_valid = client.create_generators()
gen_train, gen_valid = bb.create_generators(gen_train, gen_valid, msk='tumor')

Visualize within the following code:

In [ ]:
# --- Show first iteration
xs, ys = next(gen_train)
msk = bb.convert_box_to_msk(box=ys, apply_deltas=False)
imshow(xs['dat'][:, 0], msk[:, 0], figsize=(12, 12))

### Inputs

Similar to the above, to generate modified `inputs`, pass the existing original `inputs` into the `bb.get_inputs(...)` method:

In [ ]:
# --- Create inputs
inputs = client.get_inputs(Input)
inputs = bb.get_inputs(inputs, Input)

# Feature Pyramid Network

Now that the inputs and outputs of the CNN have been defined, the goal is to implement a network architecture that is able to perform the desired mapping via a feature pyramid network. The contracting arm of a FPN architecture is nonspecific and can be implemented using any standard architecture.

Let us define the contracting arm as follows:

### Contracting arm

In [ ]:
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))

# --- Define contracting layers
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(24, conv2(24, l2))
l4 = conv1(32, conv2(32, l3))
l5 = conv1(48, conv2(48, l4))

**Checkpoint**: The most important part here is simply to ensure that the deepest layer is at least the same size (or smaller) than the smallest anchor grid size needed for the box network. How can we confirm this?

### Expanding arm

To create layers of the expanding arm of the FPN, two special new operations must be defined.

![Box Parameterization](https://raw.githubusercontent.com/peterchang77/dl_tutor/master/cs190/spring_2020/notebooks/box_localization/pngs/fpn.png)

First, to upsample an FPN feature map, a simple parameterless interpolation is used. The corresponding Tensorflow class is the `layers.UpSampling3D(...)` object. Let us define the corresponding lambda function here:

In [ ]:
# --- Define zoom
zoom = lambda x : layers.UpSampling3D(
    size=(1, 2, 2))(x)

Additionally, in order to add the corresponding contract arm layer, recall that a 1 x 1 x 1 convolution must be used to **match feature map channels (filters)**. Recall that all FPN output maps must have the same identical number of channels (in our case, 64). Let us define the corresponding lambda function here:

In [ ]:
# --- Define 1 x 1 x 1 projection
proj = lambda filters, x : layers.Conv3D(
    filters=filters,
    strides=1,
    kernel_size=(1, 1, 1),
    padding='same',
    kernel_initializer='he_normal')(x)

Now we are ready to define the expanding layers. Recall that we only need to create the required anchor grid sizes as defined above:

* c4: 15 x 15
* c3: 30 x 30

Once these have been created, there is no need to define more expansions.

In [ ]:
# --- Define expanding layers
l6 = proj(64, l5)
l7 = conv1(64, zoom(l6) + proj(64, l4))

# RetinaNet

Now that the FPN backbone has been created, we must finalize the feature maps to perform the classification and regression tasks necessary to predict boxes. There are many ways to implement this final mapping; we will derive a method based off of the approach described in the RetinaNet paper. This implementation is quite simple (most of the "power" lies in the focal loss function) and simply requires that at each feature map resolution, a classifier head is created to perform the necessary classification and regression tasks.

![Box Parameterization](https://raw.githubusercontent.com/peterchang77/dl_tutor/master/cs190/spring_2020/notebooks/box_localization/pngs/retinanet.png)

In the original RetinaNet paper, four convolutional blocks are used, however in the context of medical imaging problems (e.g. less data, more homogenous predictions), we will just use two blocks.

In [ ]:
# --- Determine filter sizes
logits = {}
K = 1
A = 9

# --- C3
c3_cls = conv1(64, conv1(64, l7))
c3_reg = conv1(64, conv1(64, l7))
logits['cls-c3'] = layers.Conv3D(filters=(A * K), name='cls-c3', **kwargs)(c3_cls)
logits['reg-c3'] = layers.Conv3D(filters=(A * 4), name='reg-c3', **kwargs)(c3_reg)

# --- C4
c4_cls = conv1(64, conv1(64, l6))
c4_reg = conv1(64, conv1(64, l6))
logits['cls-c4'] = layers.Conv3D(filters=(A * K), name='cls-c4', **kwargs)(c4_cls)
logits['reg-c4'] = layers.Conv3D(filters=(A * 4), name='reg-c4', **kwargs)(c4_reg)

At last, the model can be formally created:

In [ ]:
# --- Create model
model = Model(inputs=inputs, outputs=logits)

# Compiling Model

There are several modifications needed to compile this box model compared to the standard approaches. 

### Focal loss

The first modification is the use of a specific **focal loss**. As you recall, the **focal loss** function gradually titrates the contribution of any given prediction such that more confident correct predictions over time become weighted less than incorrect predictions.

Focal loss is not a default loss function built into the standard Tensorflow 2.0 / Keras library. Accordingly, a custom function has been written for use as part of the `jarvis-md` library. It is implemented as follows:

```python
def focal_sigmoid_ce(weights=1.0, scale=1.0, gamma=2.0, alpha=0.25):
    """
    Method to implement focal sigmoid (binary) cross-entropy loss

    """
    def focal_sigmoid_ce(y_true, y_pred):

        # --- Calculate standard cross entropy with alpha weighting
        loss = tf.nn.weighted_cross_entropy_with_logits(
            labels=y_true, logits=y_pred, pos_weight=alpha)

        # --- Calculate modulation to pos and neg labels 
        p = tf.math.sigmoid(y_pred)
        modulation_pos = (1 - p) ** gamma
        modulation_neg = p ** gamma

        mask = tf.dtypes.cast(y_true, dtype=tf.bool)
        modulation = tf.where(mask, modulation_pos, modulation_neg)

        return tf.math.reduce_sum(modulation * loss * weights * scale)

    return focal_sigmoid_ce
```

A custom implementation of the focal sigmoid cross-entropy loss can be invoked as follows:  

```python
# --- Create custom focal loss function
custom.focal_sigmoid_ce()
```

### Huber loss

For box regression tasks, it is common to use a combination of both L1 and L2 type losses. Most commonly the desired effect is to use an L1 loss early in training (when the loss values are large) and to transition to a smoother L2 loss as the algorithm converges. One such implementation of this smooth regression loss function is the Huber loss. 

A custom variant of the Huber loss can be invoked as follows:

```python
# --- Create custom Huber loss function
custom.sl1()
```

### Masked loss functions

As you recall, although boxes classification and regression ground truth values are calculated for **every box** per image, only a subset of boxes are used for algorithm training:

* `cls` network: only boxes with IoU > 0.5 (positive) and IoU < 0.2 (negative)
* `reg` network: only boxes that correspond to a positive classification

To account for this, a mask is passed into the custom loss functions to remove the contribution of boxes that should be ignored based on the criteria above. The itself is created by the same generator that yields input data for the model.

To use, simply invoke by passing the desired mask into the function initializer:

```python
# --- Create custom focal loss function
custom.focal_sigmoid_ce(inputs['cls-c3-msk'])
```

### Box metrics

Finally, to keep track of classification performance, the use of the standard accuracy metric is suboptimal as the number of correct box predictions will quickly saturate to 100% (as the number of negative boxes >> number of positive boxes). In fact generally speaking, any metric that tracks performance of negative box predictions gtend not be very useful.

Instead, consider the use of **sensitivity** and **PPV**:

* sensitivity (recall): TP / (TP + FN)
* positive predictive value (precision): TP / (TP + FP)

A custom variant of both sensitivity and PPV metrics for binary cross-entropy loss can be invoked as follows:

```python
# --- Create sensivity and PPV metrics
custom.sigmoid_ce_sens()
custom.sigmoid_ce_ppv()
```

### Compiling

Putting this all together:

In [ ]:
# --- Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'cls-c3': custom.focal_sigmoid_ce(inputs['cls-c3-msk']),
        'cls-c4': custom.focal_sigmoid_ce(inputs['cls-c4-msk']),
        'reg-c3': custom.sl1(inputs['reg-c3-msk']),
        'reg-c4': custom.sl1(inputs['reg-c4-msk']),
        },
    metrics={
        'cls-c3': [custom.sigmoid_ce_sens(), custom.sigmoid_ce_ppv()],
        'cls-c4': [custom.sigmoid_ce_sens(), custom.sigmoid_ce_ppv()]},
    experimental_run_tf_function=False)

# Training

### In-memory data

For moderate sized datasets which are too large to fit into immediate hard-drive cache, but small enough to fit into RAM memory, it is often times a good idea to first load all training data into RAM memory for increased speed of training. The `client` can be used for this purpose as follows:

In [ ]:
# --- Load data into memory for faster training
client.load_data_in_memory()

Now, let us train the model:

In [ ]:
# --- Train model
model.fit(
    x=gen_train, 
    steps_per_epoch=500, 
    epochs=16,
    validation_data=gen_valid,
    validation_steps=500,
    validation_freq=4,
    use_multiprocessing=True)

# Evaluation

To test the trained model, the following steps are required:

* load data
* use `model.predict(...)` to obtain logit scores
* use `BoundingBox` object to convert predictions to anchors or masks
* compare prediction with ground-truth
* serialize in Pandas DataFrame

Recall that the generator used to train the model simply iterates through the dataset randomly. For model evaluation, the cohort must instead be loaded manually in an orderly way. For this tutorial, we will create new **test mode** data generators, which will simply load each example individually once for testing. 

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train, test_valid = bb.create_generators(test_train, test_valid, msk='tumor')

**Important note**: although the model is trained using 2D slices, there is nothing to preclude passing an entire 3D volume through the model at one time (e.g. consider that the entire 3D volume is a single *batch* of data). In fact, typically performance metrics for medical imaging models are commonly reported on a volume-by-volume basis (not slice-by-slice). Thus, use the `expand=True` flag in `client.create_generators(...)` as above to yield entire 3D volumes instead of slices.

In [ ]:
# --- Run entire volume through model
x, y = next(test_train)
box = model.predict(x)

# --- Modification for < TF2.2
if type(box) is list:
    box = {k: l for k, l in zip(model.output_names, box)}

### Calculating IoUs

The logits are the raw predictions from the model, but to generate the corresponding boxes several post-processing steps are needed. First the positive boxes must be identified from the classification network. Then, the predicted template boxes need to be refined using the regression network:

![Box Parameterization](https://raw.githubusercontent.com/peterchang77/dl_tutor/master/cs190/spring_2020/notebooks/box_localization/pngs/regression.png)

The `BoundingBox` object can be used to perform these steps:

* `bb.convert_box_to_msk(...)`: convert box predictions into 3D mask (primarily visualization)
* `bb.convert_box_to_anc(...)`: convert box predictions into anchors (`[y0, x0, y1, x1]`) (calculate IoUs)

In [ ]:
# --- Convert to anchors
anchors, classes = bb.convert_box_to_anc(box)

**Checkpoint**: what do these anchors and classes represent?

As seen in the ground-truth overlays at the beginning of this tutorial, a number of boxes may be classified for each single ground-truth box. Accordinging, during inference a number of ground-truth boxes may be triggered. To prune this boxes, use **non-max suppression**, a technique that removes all boxes that above a certain IoU threshold with the high scoring box. 

Test several different IoU thresholds to its effect on box outputs:

In [ ]:
# --- Show various boxes at different NMS thresholds
msk = bb.convert_box_to_msk(box, iou_nms=0.01)
imshow(x['dat'][0], msk[0], figsize=(12, 12))

Now that the specific prediction boxes have been isolated, compare them with the ground truth boxes using the `bb.calculate_ious(...)` method. This function will compare a given single box with a list of many ground-truth anchors. The maximum overlap generated represents the IoU value for the given prediction box:

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train, test_valid = bb.create_generators(test_train, test_valid, msk='tumor')

ious = {
    'med': [],
    'p25': [],
    'p75': []}

for x, y in test_train:
    
    # --- Predict
    box = model.predict(x)
    if type(box) is list:
        box = {k: l for k, l in zip(model.output_names, box)}
        
    # --- Convert predictions to anchors
    anchors_pred, _ = bb.convert_box_to_anc(box)
    
    # --- Convert ground-truth to anchors
    anchors_true, _ = bb.convert_box_to_anc(y)
    
    # --- Calculate IoUs
    curr = []
    for pred, true in zip(anchors_pred, anchors_true):
        for p in pred:
            iou = bb.calculate_ious(box=p, anchors=true)
            if iou.size > 0:
                curr.append(np.max(iou))
            else: 
                curr.append(0)
    
    if len(curr) == 0:
        curr = [0]
        
    ious['med'].append(np.median(curr))
    ious['p25'].append(np.percentile(curr, 25))
    ious['p75'].append(np.percentile(curr, 75))
    
ious = {k: np.array(v) for k, v in ious.items()}

### Running evaluation

In [ ]:
# --- Define columns
df = pd.DataFrame(index=np.arange(ious['med'].size))
df['iou_median'] = ious['med']
df['iou_p-25th'] = ious['p25']
df['iou_p-75th'] = ious['p75']

# --- Print accuracy
print(df['iou_median'].median())
print(df['iou_p-25th'].median())
print(df['iou_p-75th'].median())

## Saving and Loading a Model

After a model has been successfully trained, it can be saved and/or loaded by simply using the `model.save()` and `models.load_model()` methods. 

In [ ]:
# --- Serialize a model
model.save('./box_localization.hdf5')

In [ ]:
# --- Load a serialized model
del model
model = models.load_model('./box_localization.hdf5', compile=False)